In [ ]:
# !pip install h3
!pip install haversine

In [15]:
import pandas as pd
import h3
from haversine import haversine

# Region View

## Region - region (distance)

Gerar coluna 'h3_cell' de acordo com a latitude e longitude do check-in. Essa coluna será responsável por representar o hexágono h3 (região) daquela coordenada.

Calcular para cada região presente ('h3_cell'), a distância dela até as demais regiões presentes
  - pesquisar como calcular a distância entre as regiões h3
  - para cada região, teremos um vetor de distâncias
  - ao final, juntar todos os vetores em uma matriz

**Entrada:**
latitude e longitude dos check-ins

**Saída:**
Matriz de adjacência entre as regiões

In [7]:
checkins = pd.read_csv('checkins.csv')

In [6]:
def generate_h3_cell(row):
    lat = row['latitude']
    lon = row['longitude']
    resolution = 9

    h3_cell = h3.geo_to_h3(lat, lon, resolution)
    return h3_cell

In [12]:
checkins['h3_cell'] = (
    checkins
    .apply(generate_h3_cell, axis=1)
)

checkins['h3_cell'].nunique()

2181

Utilizando a resolução 9, temos 2181 células h3.

Fazer um gráfico mapeando os checkins no estado de NY e fazer outro gráfico mapeando as tesselações no estado de NY, para validarmos se estamos pegando um número ideal de resolução

In [17]:
def h3_centroid_distance(cell_h3_1, cell_h3_2):
    centroid_1 = h3.h3_to_geo(cell_h3_1)
    centroid_2 = h3.h3_to_geo(cell_h3_2)
    
    distance_meters = haversine(centroid_1, centroid_2) * 1000  # Converta para metros
    return distance_meters

In [18]:
cell_h3_1 = '892a100d673ffff'
cell_h3_2 = '892a100d2c3ffff'

distance = h3_centroid_distance(cell_h3_1, cell_h3_2)
print(f'Distância entre os centroides: {distance:.2f} metros')

Distância entre os centroides: 1264.28 metros


## Region - POI (quantity)

Agrupar os dados a partir da coluna 'h3_cell' e calcular a quantidade de POIs rotulados com as 7 categorias presentes naquela célula h3.

Construir uma matriz contendo em cada linha a região e em cada coluna a quantidade de 1 categoria naquela região h3.

**Entrada:**

**Saída:**